# 홍익대학교 컴퓨터공학과
# 2021년 1학기 인공지능 프로젝트 <사기 결제 탐지>
팀원(분반) : B511198 조일우(1) / B611125 유민호(2) / B511156 이치현(2)

In [25]:
# Must Mount Google drive with "/Share/CreditCard_data/" Directory.
# x_train(Feature Data) : /content/drive/MyDrive/Share/CreditCard_Data/X_train.csv
# y_train(Label Data) : /content/drive/MyDrive/Share/CreditCard_Data/y_train.csv
# x_test(Feature Data) : /content/drive/MyDrive/Share/CreditCard_Data/X_test.csv
# y_test(Label Data) : /content/drive/MyDrive/Share/CreditCard_Data/y_test.csv

# 참고
# 1. TensorFlow Keras Tutorials : https://www.tensorflow.org/tutorials/keras/classification?hl=ko
# 2. TensorFlow Keras save and load Tutorials : https://www.tensorflow.org/guide/keras/save_and_serialize?hl=ko
# 3. Try and Except : https://wikidocs.net/30
# 4. Keras custom class Tutorials : https://bit.ly/3f8iE6v
# 5. DNN example : https://corock.tistory.com/451
# 6. Keras Precision, Recall, f1-score Code : https://m.blog.naver.com/PostView.naver?blogId=wideeyed&logNo=221226716255
# 7. sklearn.metircs : https://datascienceschool.net/03%20machine%20learning/09.04%20%EB%B6%84%EB%A5%98%20%EC%84%B1%EB%8A%A5%ED%8F%89%EA%B0%80.html

In [26]:
# import libs.
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler

'\n# Check the version\nprint("Tensor Version : " + str(tf.__version__))\nprint("Tensor Version : " + str(tf.keras.__version__))\n'

In [27]:
# Data Links
x_tr = '/content/drive/MyDrive/Share/CreditCard_Data/X_train.csv'
y_tr = '/content/drive/MyDrive/Share/CreditCard_Data/y_train.csv'

x_te = '/content/drive/MyDrive/Share/CreditCard_Data/X_test.csv'
y_te = '/content/drive/MyDrive/Share/CreditCard_Data/y_test.csv'

# Make dataframes with datasets.
df_x = pd.read_csv(x_tr, index_col=0)
df_y = pd.read_csv(y_tr, index_col=0)
df_xt = pd.read_csv(x_te, index_col=0)
df_yt = pd.read_csv(y_te, index_col=0)

'''
#drop time
df_x.drop(['Time'], axis=1)
df_xt.drop(['Time'], axis=1)
'''

# Normalization
sc = StandardScaler()
df_x = sc.fit_transform(df_x)
df_xt = sc.fit_transform(df_xt)

# Oversampling the train data
smote = SMOTE(random_state=11)
df_x_s,df_y_s = smote.fit_sample(df_x,df_y)

# Undersamping the test data
df_xt_s, df_yt_s = RandomUnderSampler(random_state=0).fit_resample(df_x, df_y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [28]:
# Generate model function
def makeModel(input_dime, output_dime, layer_depth=10):
  model = Sequential()
  # Input layer
  model.add(Dense(units=input_dime, kernel_initializer='uniform', input_dim=input_dime, activation="relu", name="input_Layer"))
  # Hidden Layers
  for x in range(layer_depth):
    model.add(Dense(units=16, kernel_initializer='uniform', activation="relu", name="hidden_layer_{:02}".format(x)))
    model.add(Dropout(0.25)) # Set Dropout Ratio to X
  # Output Layer
  model.add(Dense(output_dime, kernel_initializer='uniform', activation="sigmoid", name="output_Layer"))
  # Compile the model
  adam = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(loss=keras.losses.binary_crossentropy, optimizer=adam, metrics=['accuracy',tf.keras.metrics.Precision(name='Precision'),tf.keras.metrics.Recall(name='Recall')])

  return model

In [39]:
# Set Features, Hidden layers, and an output
input_features = df_x.shape[1]
num_layers = 5
output = df_y.shape[1] # {0, 1}

# Set fit() Arguments
bat_size = 100
epch = 3

# change summary view
# 0 turns on the summary // 1 turns off the summary
show_summary = 1

# Fit the model with the train dataset
model = makeModel(input_features, output, num_layers)
history = model.fit(df_x_s, df_y_s, batch_size=bat_size, epochs=epch)

# Show the model's summary
if show_summary == 0:
  model.summary()

# Evaluate the model with the test dataset
print("Model Evaluation")
result = {0,}
results = model.evaluate(df_xt_s, df_yt_s, batch_size=5)

# Print Metrics
print('== <Results> ==\n', 'Accuracy ', results[1] * 100, '%\n')
f1score = 2 * (results[2]*results[3])/(results[2]+results[3])
print('Precision \t', results[2], '\n')
print('Recall \t\t', results[3], '\n')
print('f1-score \t', f1score, '\n')
print('===============\n')

Epoch 1/3
3184/3184 [==============================] - 10s 2ms/step - loss: 0.6932 - accuracy: 0.4998 - Precision: 0.4944 - Recall: 0.2823
Epoch 2/3
3184/3184 [==============================] - 8s 2ms/step - loss: 0.6932 - accuracy: 0.5011 - Precision: 0.5003 - Recall: 0.4670
Epoch 3/3
3184/3184 [==============================] - 8s 2ms/step - loss: 0.6932 - accuracy: 0.5001 - Precision: 0.5006 - Recall: 0.4704
Model Evaluation
118/118 [==============================] - 1s 1ms/step - loss: 0.6932 - accuracy: 0.5000 - Precision: 0.5003 - Recall: 0.4891
== <Results> ==
 Accuracy  50.0 %

Precision 	 0.5002939701080322 

Recall 		 0.4890948534011841 

f1-score 	 0.4946310290823799 


